**CS560 - Algorithms and Their Analysis**
<br>
Date: **30 April 2021**
<br>

Title: **Seminar 12**
<br>
Speaker: **Dr. Shota Tsiskaridze**


Bibliography:
<br>
 **Chapter 24**. Cormen, Thomas H. and Leiserson, Charles Eric and Rivest, Ronald Linn and Stein, Clifford Seth, *Introduction to Algorithms, 3rd Edition*, MIT Press, 2009
 


<h1 align="center">Single-Source Shortest Paths</h1>

<h3 align="center">Motivation</h3>

- **Professor Patrick** wishes to find the shortest possible route from **Phoenix** to **Indianapolis**.


- Given a **road map** of the **United States** on which the distance between each pair of adjacent intersections is marked.


- How can she **determine** this **shortest route**?

- we can model the **road map** as a **graph** $G = (V, E)$: 

  **vertices** represent **intersections**, 
  
  **edges** represent **road segments** between intersections, 
  
  **edge weights** represent road **distances**. 
  

-  Our **goal** is to **find a shortest path** from a given intersection in **Phoenix** to a given intersection in **Indianapolis**.

<h3 align="center">Problem Statement</h3>

- In a **shortest-paths problem**, we are given a **weighted**, **directed graph** $G =(V, E)$, **weight function** $w : E \rightarrow \mathbb{R}$ mapping edges to real-valued weights. 


- The **weight** $w(o)$ of path $p = \left \langle v_0, v_1, ..., v_k \right \rangle$ is the **sum of the weights** of its constituent **edges**:

  $$w(p) = \sum_{i=1}^{k} w(v_{i-1}, v_i).$$
  
- We **define** the **shortest-path weight** $\delta (u, v)$ from $u$ to $v$ by:

  $$\delta(u,v) = 
  \left\{\begin{matrix}
  \min \{w(p): u \leadsto^{p} v\} & \text{ if there is a path from} u \text{ to  } v ,\\
  \infty & \text{ otherwise.}
  \end{matrix}\right.$$
  
  
- A **shortest path** from vertex $u$ to vertex $v$ is then defined as **any path** $p$ with **weight** $w(p) = \delta (u,v)$.


- **Note**:

  Edge **weights** can represent **metrics** other than **distances**, such as **time**, **cost**, **penalties**, or **any other quantity** that accumulates linearly along a path.
  
   The **breadth-first-search algorithm** from is a **shortest-paths algorithm** that works on **unweighted graphs**.

<h3 align="center">Variants</h3>

- We shall focus on the **single-source shortest-paths problem**: 

  Given a graph $G = (V, E)$, we want to **find a shortest path** from a given **source vertex** $s \in V$ to each vertex $u \in V$. 
  
  
- The **algorithm** for the **single-source problem** can **solve many other problems**.


- **Single-destination shortest-paths problem**: 

  **Find a shortest path** to a given **destination** vertex $t$ from each vertex $v$. 
  
  By reversing the direction of each edge in the graph, we can reduce this problem to a single-source problem.
  
  
- **Single-pair shortest-path problem**: 

  **Find a shortest path** from $u$ to $v$ for given **vertices** $u$ and $v$.
  
  If we solve the single-source problem with source vertex $u$, we solve this problem also. 
  
  Moreover, all known algorithms for this problem have the same worst-case asymptotic running time as the best single-source algorithms.
  
  
- **All-pairs shortest-paths problem**: 

  **Find a shortest path** from $u$ to $v$ for every pair of vertices $u$ and $v$. 
  
  Although we can solve this problem by running a **singlesource algorithm** once from each vertex, we usually can solve it faster.

<h3 align="center">Optimal Substructure of a Shortest Path</h3>

- **Shortest-paths algorithms** typically **rely** on the **property** that a **shortest path** between two vertices **contains other shortest paths** within it.


- **Lemma (Subpaths of shortest paths are shortest paths)**:

  Given a **weighted**, **directed graph** $G = (V, E)$ with weight function $w :E \rightarrow \mathbb{R}$.
  
  Let $p = \left \langle v_0, v_1, ..., v_k \right \rangle$ be a shortest path from vertex $v_0$ to vertex $v_k$.
  
  For any $i$ and $j$ such that $0 \leq i \leq j \leq k$, let $p_{ij} = \left \langle v_i, v_{i+1}, ..., v_j \right \rangle$ be the subpath of $p$ from vertex $v_i$ to vertex $v_j$. 
  
  Then, $p_{ij}$ is a **shortest path** from $v_i$ to $v_j$.

<h3 align="center">Negative-Weight Edges</h3>

- **Some instances** of the single-source shortest-paths problem **may include edges** whose **weights are negative**. 


- If the graph $G = (V, E)$ contains **no negativeweight cycles** reachable from the **source** $s$, then for all $v \in V$, the shortest-path weight $\delta(s, v)$ remains well defined, even if it has a negative value. 


- If the graph contains a **negative-weight cycle** reachable from $s$, however, shortest-path weights are **not well defined**. 

  If there is a **negativeweight cycle** on some path from $s$ to $v$, we define $\delta (s, v) = - \infty$.
  
  <img src="images/L12_NG.png" width="800" alt="Example" />

  

<h3 align="center">Cycles</h3>

- Can a **shortest path** contain a **cycle**?


- As we have just seen, it **cannot contain** a **negative-weight cycle**. 


- **Nor can it contain** a **positive-weight cycle**, since removing the cycle from the path produces a path with the same source and destination vertices and a lower path weight. 

  That is, if $p = \left \langle v_0, v_1, ..., v_k \right \rangle$ is a path and $q = \left \langle v_i, v_{i+1}, ..., v_j \right \rangle$ is a positive-weight cycle on this path (so that $v_i = v_j$ and $w(q) > 0 $), then the path $r = \left \langle v_0, v_1, ..., v_i, v_{j+1}, ...,  v_k \right \rangle$ has weight $w(r) = w(p) - w(q)$, and so $p$ cannot be a shortest path from $v_0$ to $v_k$.
  
  
- That leaves only **0-weight cycles**, that we can remove from any path to produce another path whose weight is the same.


- Therefore, **without loss of generality** we can assume that when we are finding **shortest paths**, they have **no cycles**. 

  Since any acyclic path in a graph $G = (V, E)$ contains at most $|V|$ distinct vertices, it also contains at most $|V|-1j$ edges. 
  
  Thus, we can **restrict our attention** to **shortest paths of at most $|V| - 1$ edges**.

<h3 align="center">Relaxation</h3>

- For each vertex $v\in V$, we maintain an attribute $v.d$, which is an upper bound on the weight of a shortest path from source $s$ to $v$.


- We call $v.d$ a **shortest-path estimate**. 


- We **initialize** the shortest-path estimates and predecessors by the following $\Theta(V)$-time procedure:

  <img src="images/L12_Pseudocode_initialize.png" width="400" alt="Example" />
  
  After initialization, we have $v.\pi = NIL$ for all $v \in V$, $s.d = 0$ and $v.d = \infty$ for $v \in V - \{s\}$.

- The process of **relaxing** an edge $(u,v)$ consists of testing whether we can improve the shortest path to $v$ found so far by going through $u$.

  If so, updating $v.d$ and $v.\pi$.


- A **relaxation step** may decrease the value of the shortest-path estimate $v.d$ and update $v$’s predecessor attribute $v.\pi$. 


- The following code performs a relaxation step on edge $(u,v)$ in $O(1)$ time:

  <img src="images/L12_Pseudocode_Relax.png" width="400" alt="Example" />



<h3 align="center">The Bellman-Ford Algorithm</h3>

- The **Bellman-Ford algorithm** solves the **single-source shortest-paths problem** in the **general case** in which **edge weights may be negative**. 


- Given a **weighted**, **directed graph** $G = (V,E)$ with **source** $s$ and **weight function** $w: E \rightarrow \mathbb{R}$.


- The Bellman-Ford algorithm **returns a boolean value** indicating **whether or not** there is a **negative-weight cycle** that is reachable from the source. 

  If **there is** such a cycle, the **algorithm indicates that no solution exists**. 
  
  If **there is no** such cycle, the algorithm **produces the shortest paths** and their weights.
  
  
- The **algorithm relaxes edges**, progressively **decreasing an estimate** $v.d$ on the weight of a shortest path from the source $s$ to each vertex $v \in V$ **until** it **achieves the actual shortest-path weight** $\delta(s,v)$. 


- The algorithm returns $TRUE$ if and only if the **graph contains no negative-weight cycles** that are reachable from the source:

  <img src="images/L12_Pseudocode_Bellman_Ford.png" width="500" alt="Example" />

- The Bellman-Ford algorithm **runs in time** $O(VE)$, since:

   The initialization in **line 1** takes $\Theta(V)$ time;
  
   Each of the $|V| - 1$ passes over the edges in **lines 2–4** takes $\Theta(E)$ time;
  
   The **for** loop of **lines 5–7** takes $O(E)$ time.


- The execution of the Bellman-Ford algorithm is shown below:

  <img src="images/L12_Execution_Bellman_Ford.png" width="1000" alt="Example" />



In [ ]:
# Python3 program for Bellman-Ford's single source  
# shortest path algorithm.  
  
# Class to represent a graph  
class Graph:  
  
    def __init__(self, vertices):  
        self.V = vertices # No. of vertices  
        self.graph = []  
  
    # function to add an edge to graph  
    def addEdge(self, u, v, w):  
        self.graph.append([u, v, w])  
          
    # utility function used to print the solution  
    def printArr(self, dist):  
        print("Vertex Distance from Source")  
        for i in range(self.V):  
            print("{0}\t\t{1}".format(i, dist[i]))  
      
    # The main function that finds shortest distances from src to  
    # all other vertices using Bellman-Ford algorithm. The function  
    # also detects negative weight cycle  
    def BellmanFord(self, src):  
  
        # Step 1: Initialize distances from src to all other vertices  
        # as INFINITE  
        dist = [float("Inf")] * self.V  
        dist[src] = 0
  
  
        # Step 2: Relax all edges |V| - 1 times. A simple shortest  
        # path from src to any other vertex can have at-most |V| - 1  
        # edges  
        for _ in range(self.V - 1):  
            # Update dist value and parent index of the adjacent vertices of  
            # the picked vertex. Consider only those vertices which are still in  
            # queue  
            for u, v, w in self.graph:  
                if dist[u] != float("Inf") and dist[u] + w < dist[v]:  
                        dist[v] = dist[u] + w  
  
        # Step 3: check for negative-weight cycles. The above step  
        # guarantees shortest distances if graph doesn't contain  
        # negative weight cycle. If we get a shorter path, then there  
        # is a cycle.  
  
        for u, v, w in self.graph:  
                if dist[u] != float("Inf") and dist[u] + w < dist[v]:  
                        print("Graph contains negative weight cycle") 
                        return
                          
        # print all distance  
        self.printArr(dist)
        
g = Graph(5)  
g.addEdge(0, 1,  6)  # s -> t
g.addEdge(0, 3,  7)  # s -> y
g.addEdge(1, 2,  5)  # t -> x
g.addEdge(1, 3,  8)  # t -> y
g.addEdge(1, 4, -4)  # t -> z
g.addEdge(2, 1, -2)  # x -> t
g.addEdge(3, 2, -3)  # y -> x
g.addEdge(3, 4,  9)  # y -> z
g.addEdge(4, 0,  2)  # z -> s
g.addEdge(4, 2,  7)  # z -> x

  
# Print the solution  
g.BellmanFord(0)  

<h3 align="center">Single-Source Shortest Paths in Directed Acyclic Graphs</h3>

- The **Pseoudocode** for `Dag-Shortest-Path` is as follows:

  <img src="images/L12_Pseudocode_DAG.png" width="600" alt="Example" />
  
  - The execution of the **DAG-Shortest-Path** algorithm is shown below:

  <img src="images/L12_Execution_DAG.png" width="1000" alt="Example" />

In [1]:
# Python program to find single source shortest paths 
# for Directed Acyclic Graphs Complexity :OV(V+E) 
from collections import defaultdict 
  
# Graph is represented using adjacency list. Every 
# node of adjacency list contains vertex number of 
# the vertex to which edge connects. It also contains 
# weight of the edge 
class Graph: 
    def __init__(self,vertices): 
  
        self.V = vertices # No. of vertices 
  
        # dictionary containing adjacency List 
        self.graph = defaultdict(list) 
  
    # function to add an edge to graph 
    def addEdge(self,u,v,w): 
        self.graph[u].append((v,w)) 
  
  
    # A recursive function used by shortestPath 
    def topologicalSortUtil(self,v,visited,stack): 
  
        # Mark the current node as visited. 
        visited[v] = True
  
        # Recur for all the vertices adjacent to this vertex 
        if v in self.graph.keys(): 
            for node,weight in self.graph[v]: 
                if visited[node] == False: 
                    self.topologicalSortUtil(node,visited,stack) 
  
        # Push current vertex to stack which stores topological sort 
        stack.append(v) 
  
  
    ''' The function to find shortest paths from given vertex. 
        It uses recursive topologicalSortUtil() to get topological 
        sorting of given graph.'''
    def dagShortestPath(self, s): 
  
        # Mark all the vertices as not visited 
        visited = [False]*self.V 
        stack =[] 
  
        # Call the recursive helper function to store Topological 
        # Sort starting from source vertice 
        for i in range(self.V): 
            if visited[i] == False: 
                self.topologicalSortUtil(s,visited,stack) 
  
        # Initialize distances to all vertices as infinite and 
        # distance to source as 0 
        dist = [float("Inf")] * (self.V) 
        dist[s] = 0
  
        # Process vertices in topological order 
        while stack: 
  
            # Get the next vertex from topological order 
            i = stack.pop() 
  
            # Update distances of all adjacent vertices 
            for node,weight in self.graph[i]: 
                if dist[node] > dist[i] + weight: 
                    dist[node] = dist[i] + weight 
  
        # Print the calculated shortest distances 
        for i in range(self.V): 
            print ("%d" %dist[i]) if dist[i] != float("Inf") else  "Inf" , 
  
  
g = Graph(6)
g.addEdge(0, 1,  5)  # r -> s
g.addEdge(0, 2,  3)  # r -> t
g.addEdge(1, 2,  2)  # s -> t
g.addEdge(1, 3,  6)  # s -> x
g.addEdge(2, 3,  7)  # t -> x
g.addEdge(2, 4,  4)  # t -> y
g.addEdge(2, 5,  2)  # t -> z
g.addEdge(3, 4, -1)  # x -> y
g.addEdge(3, 5,  1)  # x -> z
g.addEdge(4, 5, -2)  # y -> z
  
# source = 1 
s = 1
  
print ("Following are shortest distances from source %d " % s) 
g.dagShortestPath(s)

Following are shortest distances from source 1 
0
2
6
5
3


<h3 align="center">Dijkstra’s Algorithm</h3>

- The **Pseoudocode** for `Dijkstra` is as follows:


  <img src="images/L12_Pseudocode_Dijekstra.png" width="500" alt="Example" />


- The execution of the **Dijekstra** algorithm is shown below:

  <img src="images/L12_Execution_Dijekstra.png" width="1000" alt="Example" />

In [ ]:
import sys 
   
class Graph(): 
   
    def __init__(self, vertices): 
        self.V = vertices 
        self.graph = [[0 for column in range(vertices)]  
                    for row in range(vertices)] 
   
    def printSolution(self, dist): 
        print ("Vertex tDistance from Source") 
        for node in range(self.V): 
            print (node, ": dist =", dist[node]) 
   
    # A utility function to find the vertex with  
    # minimum distance value, from the set of vertices  
    # not yet included in shortest path tree 
    def minDistance(self, dist, sptSet): 
   
        # Initilaize minimum distance for next node 
        min = sys.maxsize 
   
        # Search not nearest vertex not in the  
        # shortest path tree 
        for v in range(self.V): 
            if dist[v] < min and sptSet[v] == False: 
                min = dist[v] 
                min_index = v 
   
        return min_index 
   
    # Funtion that implements Dijkstra's single source  
    # shortest path algorithm for a graph represented  
    # using adjacency matrix representation 
    def dijkstra(self, src): 
   
        dist = [sys.maxsize] * self.V 
        dist[src] = 0
        sptSet = [False] * self.V 
   
        for cout in range(self.V): 
   
            # Pick the minimum distance vertex from  
            # the set of vertices not yet processed.  
            # u is always equal to src in first iteration 
            u = self.minDistance(dist, sptSet) 
   
            # Put the minimum distance vertex in the  
            # shotest path tree 
            sptSet[u] = True
   
            # Update dist value of the adjacent vertices  
            # of the picked vertex only if the current  
            # distance is greater than new distance and 
            # the vertex in not in the shotest path tree 
            for v in range(self.V): 
                if self.graph[u][v] > 0 and sptSet[v] == False and dist[v] > dist[u] + self.graph[u][v]:
                    dist[v] = dist[u] + self.graph[u][v] 
    
        self.printSolution(dist) 

G = Graph(5) 

           # s,  t,  x,  y,  z
G.graph = [ [0, 10,  0,  5,  0], # s
            [0,  0,  1,  2,  0], # t
            [0,  0,  0,  0,  4], # x
            [0,  3,  9,  0,  2], # y
            [0,  0,  6,  0,  0]] # z
   
G.dijkstra(0); 

<h1 align="center">End of Seminar</h1>